# Amazon SageMaker Clarify Workshop

Amazon SageMaker Clarify provides tools to help explain how machine learning (ML) models make predictions. These tools can help ML modelers and developers and other internal stakeholders understand model characteristics as a whole prior to deployment and to debug predictions provided by the model after it's deployed. Transparency about how ML models arrive at their predictions is also critical to consumers and regulators who need to trust the model predictions if they are going to accept the decisions based on them. SageMaker Clarify uses a model-agnostic feature attribution approach, which you can used to understand why a model made a prediction after training and to provide per-instance explanation during inference. The implementation includes a scalable and efficient implementation of SHAP ([see paper](https://papers.nips.cc/paper/2017/file/8a20a8621978632d76c43dfd28b67767-Paper.pdf)), based on the concept of a Shapley value from the field of cooperative game theory that assigns each feature an importance value for a particular prediction.

## Business problem

<i> "Auto insurance fraud ranges from misrepresenting facts on insurance applications and inflating insurance claims to staging accidents and submitting claim forms for injuries or damage that never occurred, to false reports of stolen vehicles.
Fraud accounted for between 15 percent and 17 percent of total claims payments for auto insurance bodily injury in 2012, according to an Insurance Research Council (IRC) study. The study estimated that between $\$5.6$ billion and $\$7.7$ billion was fraudulently added to paid claims for auto insurance bodily injury payments in 2012, compared with a range of $\$4.3$ billion to $\$5.8$ billion in 2002. </i>" [source: Insurance Information Institute](https://www.iii.org/article/background-on-insurance-fraud)

In this example, we will use an *auto insurance domain* to detect claims that are possibly fraudulent. It is a synthetic data set containing customer information and claims information. 

## Setup

First, download and upgrade some Python libraries.

In [ ]:
!python -m pip install -q awswrangler==2.2.0 imbalanced-learn==0.7.0 sagemaker==2.41.0 boto3==1.17.70

Import the libraries needed for this sample.

In [ ]:
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

import boto3
import sagemaker
from sagemaker.xgboost.estimator import XGBoost
import awswrangler

Set up SageMaker session and boto3 session variables, and specify the bucket and prefix used for storing the data and results.

In [ ]:
sagemaker_session = sagemaker.Session()
sagemaker_role = sagemaker.session.get_execution_role(sagemaker_session = sagemaker_session)
region = sagemaker_session.boto_region_name
sagemaker_boto_client = boto3.Session(region_name=region).client("sagemaker")
s3_client = boto3.client("s3", region_name=region)
bucket = sagemaker_session.default_bucket()
prefix = 'clarify-smote-workshop'

Set up more variables which will be needed for the training job and Clarify job.

In [ ]:
train_data_uri = f"s3://{bucket}/{prefix}/data/train/train.csv"
test_data_uri = f"s3://{bucket}/{prefix}/data/test/test.csv"

bias_report_1_output_path = f"s3://{bucket}/{prefix}/clarify-output/bias_1"
bias_report_2_output_path = f"s3://{bucket}/{prefix}/clarify-output/bias-2"
explainability_output_path = f"s3://{bucket}/{prefix}/clarify-output/explainability"

estimator_output_path = f"s3://{bucket}/{prefix}/training_jobs"
train_instance_count = 1
train_instance_type = "ml.m4.xlarge"

The data has already been prepared for you and is available in the 'data' folder. Let's upload it to the S3 bucket specified earlier.

In [ ]:
sagemaker_session.upload_data("data/train.csv", bucket = bucket, key_prefix = f"{prefix}/data/train")

## Data exploration

Before training a model, take a quick look at the training data below. The first column, `fraud`, is the column you want to predict, where `1` indicates a case of fraud and `0` indicates a legit case. The dataset contains columns related to the customer (e.g. `customer_gender_male` or `num_insurers_past_5_years`), related to the incident (e.g. `num_witnesses` or `incident_month`), and related to the insurance policy (e.g. `policy_deductable` or `policy_state_ca`). Notice that the data has already been cleaned and pre-processed for you. For example, the customer gender and policy states have already gone through one-hot encoding. 

In [ ]:
train_data = pd.read_csv("data/train.csv")
train_data.head()

The table above provides a sample of the data, but let's take a look at all the columns and check for null values. Using the code below, you can see that all null values have been removed from the dataset already.

In [ ]:
train_data.info(verbose=True)

## Train a model using XGBoost

Once the training dataset has been persisted in S3, you can start training a model by defining which SageMaker Estimator you'd like to use. For this guide, you will use the [XGBoost Open Source Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/xgboost/xgboost.html) to train your model. This estimator is accessed via the SageMaker SDK, but mirrors the open source version of the [XGBoost Python package](https://xgboost.readthedocs.io/en/latest/python/index.html). Any functionality provided by the XGBoost Python package can be implemented in your training script. This training script has already been provided as `xgboost_starter_script.py`. Feel free to open it and take a look.

The code below sets up the hyperparameters and the XGBoost object which uses the script.

In [ ]:
hyperparameters = {
    "max_depth": "3",
    "eta": "0.2",
    "objective": "binary:logistic",
    "num_round": "100",
}

xgb_estimator = XGBoost(
    entry_point="xgboost_starter_script.py",
    output_path=estimator_output_path,
    hyperparameters=hyperparameters,
    role=sagemaker_role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    framework_version="1.0-1",
)

Now we can run the model training using the pre-prepared dataset. The code below checks if there is an existing training job to reuse, and otherwise starts a new training job.

In [ ]:
if 'training_job_1_name' not in locals():
    xgb_estimator.fit(inputs = {'train': train_data_uri})
    training_job_1_name = xgb_estimator.latest_training_job.job_name
    %store training_job_1_name
else:
    print(f'Using previous training job: {training_job_1_name}')

Use the code below to fetch some basic information from the training job. You'll need these to create the model object.

In [ ]:
training_job_1_info = sagemaker_boto_client.describe_training_job(TrainingJobName=training_job_1_name)
training_data_s3_uri = training_job_1_info["InputDataConfig"][0]["DataSource"]["S3DataSource"]["S3Uri"]

## Check for data set bias and model bias

With SageMaker, we can check for pre-training and post-training bias. Pre-training metrics show pre-existing bias in that data, while post-training metrics show bias in the predictions from the model. Using the SageMaker SDK, we can specify which groups we want to check bias across and which metrics we'd like to show.

First, create a model object directly from the training job. By creating the model object, Clarify will be able to calculate post-training metrics along with pre-training metrics (for which no model is required).

In [ ]:
model_1_name = f"{prefix}-xgboost-pre-smote"
model_matches = sagemaker_boto_client.list_models(NameContains=model_1_name)["Models"]

if not model_matches:
    model_1 = sagemaker_session.create_model_from_job(
        name=model_1_name,
        training_job_name=training_job_1_info["TrainingJobName"],
        role=sagemaker_role,
        image_uri=training_job_1_info["AlgorithmSpecification"]["TrainingImage"],
    )
else:
    print(f"Model {model_1_name} already exists.")

Below, you set up the various configuration objects which determine how Clarify is run. To run the full Clarify job, you must un-comment the code in the cell below. Running the job will take ~15 minutes. If you wish to save time, you can view the results in the next cell after which loads a pre-generated output if no bias job was run.

In [ ]:
train_cols = awswrangler.s3.read_csv(training_data_s3_uri).columns.to_list()

clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role=sagemaker_role,
    instance_count=1,
    instance_type="ml.c4.xlarge",
    sagemaker_session=sagemaker_session,
)

bias_data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=train_data_uri,
    s3_output_path=bias_report_1_output_path,
    label="fraud",
    headers=train_cols,
    dataset_type="text/csv",
)

model_config = sagemaker.clarify.ModelConfig(
    model_name=model_1_name,
    instance_type=train_instance_type,
    instance_count=1,
    accept_type="text/csv",
)

predictions_config = sagemaker.clarify.ModelPredictedLabelConfig(probability_threshold=0.5)

bias_config = sagemaker.clarify.BiasConfig(
    label_values_or_threshold=[0],
    facet_name="customer_gender_female",
    facet_values_or_threshold=[1],
)

# un-comment the code below to run the whole job

# if 'clarify_bias_job_1_name' not in locals():

#     clarify_processor.run_bias(
#         data_config=bias_data_config,
#         bias_config=bias_config,
#         model_config=model_config,
#         model_predicted_label_config=predictions_config,
#         pre_training_methods='all',
#         post_training_methods='all')

#     clarify_bias_job_1_name = clarify_processor.latest_job.name
#     %store clarify_bias_job_1_name

# else:
#     print(f'Clarify job {clarify_bias_job_name} has already run successfully.')

Running Clarify on your dataset or model can take ~15 minutes. If you don't have time to run the job, you can view the pre-generated results included with this demo. Otherwise, you can run the job by un-commenting the code in the cell above.

In [ ]:
if "clarify_bias_job_1_name" in locals():
    s3_client.download_file(
        Bucket=bucket,
        Key=f"{prefix}/clarify-output/bias_1/analysis.json",
        Filename="clarify_output/bias_1/analysis.json",
    )
    print(f"Downloaded analysis from previous Clarify job: {clarify_bias_job_1_name}")
else:
    print(f"Loading pre-generated analysis file...")

with open("clarify_output/bias_1/analysis.json", "r") as f:
    bias_analysis = json.load(f)

results = bias_analysis["pre_training_bias_metrics"]["facets"]["customer_gender_female"][0][
    "metrics"
][1]
print(json.dumps(results, indent=4))

In this example dataset, the data is biased against females. You can double-check the class imbalance by plotting a bar graph showing the number of data samples for male customers compared to female customers.

In [ ]:
train_data.customer_gender_female.value_counts(normalize=True).plot.bar()
plt.xticks([0, 1], ["Male", "Female"]);

We will address this in the next section where we show how we mitigate this class imbalance bias. Although we are only addressing Class Imbalance as an exemplar of bias statistics, you can also take into consideration many other factors of bias. For more detail, see : [Fairness Measures for Machine Learning in Finance](https://pages.awscloud.com/rs/112-TZM-766/images/Fairness.Measures.for.Machine.Learning.in.Finance.pdf)

## Use data augmentation to reduce bias

To handle the imbalance, we can over-sample (i.e. upsample) the minority class using [SMOTE (Synthetic Minority Over-sampling Technique)](https://arxiv.org/pdf/1106.1813.pdf). After installing the imbalanced-learn module, if you receive an ImportError when importing SMOTE, then try restarting the kernel.

First, let's take a look at the exact number of rows for each gender before applying SMOTE.

In [ ]:
gender = train_data["customer_gender_female"]
gender.value_counts()

Then apply SMOTE and take a look at the new numbers. Perfectly balanced now!

In [ ]:
smote = SMOTE(random_state=42)
train_data_upsampled, gender_res = smote.fit_resample(train_data, gender)
train_data_upsampled["customer_gender_female"].value_counts()

Save the new, balanced data set to a CSV file and upload it to S3. 

In [ ]:
train_data_upsampled.to_csv("train.csv", index=False)
sagemaker_session.upload_data("train.csv", bucket = bucket, key_prefix = f"{prefix}/data/train/upsampled")
train_data_upsampled_s3_path = f"s3://{bucket}/{prefix}/data/train/upsampled/train.csv"

## Train another model using the upsampled dataset

Repeat the same code as before to train a new XGBoost model using the upsampled dataset.

In [ ]:
hyperparameters = {
    "max_depth": "3",
    "eta": "0.2",
    "objective": "binary:logistic",
    "num_round": "100",
}

xgb_estimator_upsampled = XGBoost(
    entry_point="xgboost_starter_script.py",
    hyperparameters=hyperparameters,
    role=sagemaker_role,
    instance_count=train_instance_count,
    instance_type=train_instance_type,
    framework_version="1.0-1",
)

In [ ]:
if 'training_job_2_name' not in locals():
    xgb_estimator_upsampled.fit(inputs = {'train': train_data_upsampled_s3_path})
    training_job_2_name = xgb_estimator_upsampled.latest_training_job.job_name
    %store training_job_2_name
else:
    print(f'Using previous training job: {training_job_2_name}')

## Re-check data and model bias

Now you can repeat the data model bias checks by creating a model from the new training job, and setting up Clarify.

In [ ]:
training_job_2_info = sagemaker_boto_client.describe_training_job(
    TrainingJobName=training_job_2_name
)
training_data_s3_uri_upsampled = training_job_2_info["InputDataConfig"][0]["DataSource"]["S3DataSource"]["S3Uri"]

In [ ]:
model_2_name = f"{prefix}-xgboost-post-smote"
model_matches = sagemaker_boto_client.list_models(NameContains=model_2_name)['Models']

if not model_matches:
    
    model_2 = sagemaker_session.create_model_from_job(
        name=model_2_name,
        training_job_name=training_job_2_info['TrainingJobName'],
        role=sagemaker_role,
        image_uri=training_job_2_info['AlgorithmSpecification']['TrainingImage'])
    %store model_2_name
    
else:
    
    print(f"Model {model_2_name} already exists.")

Again, you can choose to run the full Clarify job by uncommenting the code below. However, this can take up to 15 minutes to complete, so we also provide pre-generated results with this demo.

In [ ]:
clarify_processor = sagemaker.clarify.SageMakerClarifyProcessor(
    role=sagemaker_role,
    instance_count=1,
    instance_type="ml.c4.xlarge",
    sagemaker_session=sagemaker_session,
)

bias_data_config = sagemaker.clarify.DataConfig(
    s3_data_input_path=train_data_upsampled_s3_path,
    s3_output_path=bias_report_2_output_path,
    label="fraud",
    headers=train_data.columns.to_list(),
    dataset_type="text/csv",
)

model_config = sagemaker.clarify.ModelConfig(
    model_name=model_2_name,
    instance_type=train_instance_type,
    instance_count=1,
    accept_type="text/csv",
)

predictions_config = sagemaker.clarify.ModelPredictedLabelConfig(probability_threshold=0.5)

bias_config = sagemaker.clarify.BiasConfig(
    label_values_or_threshold=[0],
    facet_name="customer_gender_female",
    facet_values_or_threshold=[1],
)

# # un-comment the code below to run the whole job

# if 'clarify_bias_job_2_name' not in locals():

#     clarify_processor.run_bias(
#         data_config=bias_data_config,
#         bias_config=bias_config,
#         model_config=model_config,
#         model_predicted_label_config=predictions_config,
#         pre_training_methods='all',
#         post_training_methods='all')

#     clarify_bias_job_2_name = clarify_processor.latest_job.name
#     %store clarify_bias_job_2_name

# else:
#     print(f'Clarify job {clarify_bias_job_2_name} has already run successfully.')

Now print the results from both bias analysis results so you can easily compare.

In [ ]:
if "clarify_bias_job_2_name" in locals():
    s3_client.download_file(
        Bucket=bucket,
        Key=f"{prefix}/clarify-output/bias-2/analysis.json",
        Filename="clarify_output/bias_2/analysis.json",
    )
    print(f"Downloaded analysis from previous Clarify job: {clarify_bias_job_2_name}\n")
else:
    print(f"Loading pre-generated analysis file...\n")

with open("clarify_output/bias_1/analysis.json", "r") as f:
    bias_analysis = json.load(f)

results = bias_analysis["pre_training_bias_metrics"]["facets"]["customer_gender_female"][0][
    "metrics"
][1]
print("Results from first job")
print(json.dumps(results, indent=4))

with open("clarify_output/bias_2/analysis.json", "r") as f:
    bias_analysis = json.load(f)

results = bias_analysis["pre_training_bias_metrics"]["facets"]["customer_gender_female"][0][
    "metrics"
][1]
print("Results from second job")
print(json.dumps(results, indent=4))

You can see that the class imbalance is now 0.0, compared to 0.25387 before. This is much better. You can also plot another bar graph using the rebalanced dataset to visually check the difference.

In [ ]:
train_data_upsampled.customer_gender_female.value_counts(normalize=True).plot.bar()
plt.xticks([0, 1], ["Male", "Female"]);

## Clean up

Run the cell below to clean up the two models created in this lab.

In [ ]:
sagemaker_boto_client.delete_model(ModelName = model_1_name)
sagemaker_boto_client.delete_model(ModelName = model_2_name)